In [1]:
from pyspark.sql import SparkSession

# Create SparkSession
spark = SparkSession.builder.getOrCreate()


22/11/01 11:17:08 WARN Utils: Your hostname, kevin resolves to a loopback address: 127.0.1.1; using 192.168.1.6 instead (on interface wlp0s20f3)
22/11/01 11:17:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/01 11:17:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df = spark.read.format('csv')\
                .option('header', 'true')\
                .option('inferSchema', 'true')\
                .load('/home/kevin/Desktop/Big-Data-with-Pyspark/data/retail-data/by-day/2010-12-01.csv')

df.printSchema()
df.createOrReplaceTempView('dfTable')

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



# Converting to Spark Types
We see the lit function that converts a type in another language to its corresponding spark representation

In [3]:
from pyspark.sql.functions import lit

df.select(lit(5), lit('five'), lit(5.0))

DataFrame[5: int, five: string, 5.0: double]

# Working with Booleans

In [4]:
from pyspark.sql.functions import col

df.where(col('InvoiceNo') != 5363)\
    .select('InvoiceNo', 'Description')\
    .show(5, False)

+---------+-----------------------------------+
|InvoiceNo|Description                        |
+---------+-----------------------------------+
|536365   |WHITE HANGING HEART T-LIGHT HOLDER |
|536365   |WHITE METAL LANTERN                |
|536365   |CREAM CUPID HEARTS COAT HANGER     |
|536365   |KNITTED UNION FLAG HOT WATER BOTTLE|
|536365   |RED WOOLLY HOTTIE WHITE HEART.     |
+---------+-----------------------------------+
only showing top 5 rows



In [5]:
from pyspark.sql.functions import instr
priceFilter = col('UnitPrice') > 600
descFilter = instr(df.Description, 'POSTAGE') >= 1
df.where(df.StockCode.isin('DOT')).where(priceFilter | descFilter).show(4)

+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|   Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|   536544|      DOT|DOTCOM POSTAGE|       1|2010-12-01 14:32:00|   569.77|      null|United Kingdom|
|   536592|      DOT|DOTCOM POSTAGE|       1|2010-12-01 17:06:00|   607.49|      null|United Kingdom|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+



# Working with Numbers
To fabricate a contrived example, let us imagine that we mis-recorded the quantity in our retail dataset and the true quantity is equal to (the current quantity * the unit price) squared + 5. 

In [7]:
from pyspark.sql.functions import expr, pow

fabricatedQuantity = pow(col('quantity') * col('UnitPrice'), 2) + 5
df.select(expr('CustomerId'), fabricatedQuantity.alias('realQuantity')).show(4)

+----------+------------------+
|CustomerId|      realQuantity|
+----------+------------------+
|   17850.0|239.08999999999997|
|   17850.0|          418.7156|
|   17850.0|             489.0|
|   17850.0|          418.7156|
+----------+------------------+
only showing top 4 rows



In [8]:
from pyspark.sql.functions import lit, round, bround
df.select(round(lit('2.5')), bround(lit('2.5'))).show(2)

+-------------+--------------+
|round(2.5, 0)|bround(2.5, 0)|
+-------------+--------------+
|          3.0|           2.0|
|          3.0|           2.0|
+-------------+--------------+
only showing top 2 rows



# Working with Strings

In [10]:
from pyspark.sql.functions import initcap
df.select(initcap(col('Description'))).show(4)

+--------------------+
|initcap(Description)|
+--------------------+
|White Hanging Hea...|
| White Metal Lantern|
|Cream Cupid Heart...|
|Knitted Union Fla...|
+--------------------+
only showing top 4 rows



In [11]:
from pyspark.sql.functions import lower, upper

df.select(col('Description'), lower(col('Description')), upper(col('Description'))).show(5)

+--------------------+--------------------+--------------------+
|         Description|  lower(Description)|  upper(Description)|
+--------------------+--------------------+--------------------+
|WHITE HANGING HEA...|white hanging hea...|WHITE HANGING HEA...|
| WHITE METAL LANTERN| white metal lantern| WHITE METAL LANTERN|
|CREAM CUPID HEART...|cream cupid heart...|CREAM CUPID HEART...|
|KNITTED UNION FLA...|knitted union fla...|KNITTED UNION FLA...|
|RED WOOLLY HOTTIE...|red woolly hottie...|RED WOOLLY HOTTIE...|
+--------------------+--------------------+--------------------+
only showing top 5 rows



## Regular Expressions

In [12]:
from pyspark.sql.functions import regexp_replace
regex_string = "BLACK|WHITE|RED|GREEN|BLUE"
df.select(
    regexp_replace(col('Description'), regex_string, 'COLOR').alias('color_clean'),
    col('Description')
).show(5)

+--------------------+--------------------+
|         color_clean|         Description|
+--------------------+--------------------+
|COLOR HANGING HEA...|WHITE HANGING HEA...|
| COLOR METAL LANTERN| WHITE METAL LANTERN|
|CREAM CUPID HEART...|CREAM CUPID HEART...|
|KNITTED UNION FLA...|KNITTED UNION FLA...|
|COLOR WOOLLY HOTT...|RED WOOLLY HOTTIE...|
+--------------------+--------------------+
only showing top 5 rows



In [13]:
from pyspark.sql.functions import translate

df.select(translate(col('Description'), 'LEET', '1337'), col('Description')).show(2)

+----------------------------------+--------------------+
|translate(Description, LEET, 1337)|         Description|
+----------------------------------+--------------------+
|              WHI73 HANGING H3A...|WHITE HANGING HEA...|
|               WHI73 M37A1 1AN73RN| WHITE METAL LANTERN|
+----------------------------------+--------------------+
only showing top 2 rows



In [15]:
from pyspark.sql.functions import regexp_extract

extract_string = "(BLACK|WHITE|RED|GREEN|BLUE)"
df.select(
    regexp_extract(col('Description'), extract_string, 1).alias('color_clean'),
    col('Description')
).show(2)

+-----------+--------------------+
|color_clean|         Description|
+-----------+--------------------+
|      WHITE|WHITE HANGING HEA...|
|      WHITE| WHITE METAL LANTERN|
+-----------+--------------------+
only showing top 2 rows



# Working with Dates and Timestamps

In [16]:
from pyspark.sql.functions import current_date, current_timestamp
dateDF = spark.range(10)\
                .withColumn('today', current_date())\
                .withColumn('mow', current_timestamp())
dateDF.createOrReplaceTempView('dateTable')
dateDF.printSchema()

root
 |-- id: long (nullable = false)
 |-- today: date (nullable = false)
 |-- mow: timestamp (nullable = false)



In [17]:
from pyspark.sql.functions import date_add, date_sub

dateDF.select(date_sub(col('today'), 5), date_add(col('today'), 5)).show(1)

+------------------+------------------+
|date_sub(today, 5)|date_add(today, 5)|
+------------------+------------------+
|        2022-10-27|        2022-11-06|
+------------------+------------------+
only showing top 1 row



## Working with nulls in data

## Coalesce
This is a function that allows you to select6 the first non-null value from a set of columns by uisng the coalesce function. 

In [19]:
from pyspark.sql.functions import coalesce

df.select(coalesce(col('Description')), col('Description')).show(4)

+---------------------+--------------------+
|coalesce(Description)|         Description|
+---------------------+--------------------+
| WHITE HANGING HEA...|WHITE HANGING HEA...|
|  WHITE METAL LANTERN| WHITE METAL LANTERN|
| CREAM CUPID HEART...|CREAM CUPID HEART...|
| KNITTED UNION FLA...|KNITTED UNION FLA...|
+---------------------+--------------------+
only showing top 4 rows



## ifnull, nullif, nvl, nvl2
ifnull allows you to select the second value if the first is null, and defaults to the first
nullif returns null if the two values are equal or else returns the second if they are not
nvl returns the second value if the first is null, but defaults to the first
nvl2 returns the second value if the first is not null; otherwise, it returns the last specified value

## fill

In [20]:
df.na.fill('All values')

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: timestamp, UnitPrice: double, CustomerID: double, Country: string]

In [21]:
spark.stop()